In [1]:
import numpy as np

In [2]:
def periodic_distance(atom, nbd, box):
    '''Inputs:
        atom = coordinate of a single reference atom
        nbd = array of coordinates of multiple atoms
        box = length of the box
       Outputs:
        dis = distance between atom and nbd
        len(dis) == len(nbd)
    '''
    delta = np.abs(atom-nbd)
    delta = np.where(delta> 0.5*box, delta-box, delta)
    dis = np.sqrt((delta**2).sum(axis=-1))
    return dis

In [3]:
import periodictable

def get_atomic_mass(element_symbol):
    try:
        element = periodictable.elements.symbol(element_symbol)
        atomic_mass = element.mass
        return atomic_mass
    except ValueError as e:
        print(f"Error: {e}")
        return None


In [7]:
# Choose an x for creating (1-x)LiNbO3-xSiO2 system
frac = 0.00

# We want a total of n_mols number of molecules (sum of number of LiNbO3 and SiO2 molecules)
# If x == 0, there will be n_mols number of LiNbO3
n_mols = 20000
n_Li = int(n_mols*(1-frac))
n_Nb = n_Li

n_O = int(n_mols*(frac*2+(1-frac)*3))
n_Si = int(n_mols*frac)

# Manually choose the number of Er
n_Er = 0
n_O += int(n_Er*3/2.0)

print("Number of atoms: Li:{}, Nb:{}, O:{}, Si:{}, Er:{}".format(n_Li, n_Nb, n_O, n_Si, n_Er))
print("Total number of atoms: ", np.sum([n_Li, n_Nb, n_O, n_Si, n_Er]))

Number of atoms: Li:20000, Nb:20000, O:60000, Si:0, Er:0
Total number of atoms:  100000


In [12]:
contents = {'Li':n_Li,'Nb':n_Nb,'O':n_O}#,'Si':n_Si,'Er':n_Er} #LNS5

total_mass = 0.0
for key,num in contents.items():
    total_mass += num*get_atomic_mass(key)

In [13]:
import numpy as np


density = 4.20   #g/cm3
box = np.cbrt((total_mass*1.66054)/(density))
print("Box size:", box, "A.")

Box size: 105.34472588464038 A.


__Take the coordinates of the supercell of the seed to be inserted and then displace it to the center of the box__


In [55]:
seed = np.loadtxt("LiNbO3_4x4x1.poscar", skiprows=8)

mean_x = np.mean(seed[:,0])
mean_y = np.mean(seed[:,1])
mean_z = np.mean(seed[:,2])
seed_x = [i+(box*0.5-mean_x) for i in seed[:,0]]
seed_y = [i+(box*0.5-mean_y) for i in seed[:,1]]
seed_z = [i+(box*0.5-mean_z) for i in seed[:,2]]


In [57]:
num_atoms = list(contents.values())

In [58]:
natoms_summed = [sum(num_atoms[:i+1]) for i in range(len(num_atoms))]

print(natoms_summed)


[20000, 40000, 100000]


In [46]:
from tqdm import tqdm
import random as ran
import numpy as np



cut_off = 1.5                 #minimum approach distance
center=[box*0.5, box*0.5, box*0.5]
# Initialize the lists with the coordinates of the seeds loaded above
rx,ry,rz = seed_x[:], seed_y[:], seed_z[:]
print ("Now creating the random coordinates.................")


print ("Now placing Li!!!")
for count in tqdm(range(natoms_summed[0]), desc="Processing", ncols=100):
    x,y,z = box*ran.random(),box*ran.random(),box*ran.random()
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off+0.5 for value in dist):
            checker = "Passed"
        else:
            x,y,z = box*ran.random(),box*ran.random(),box*ran.random()
    rx.append(x)
    ry.append(y)
    rz.append(z)


Now creating the random coordinates.................
Now placing Li!!!


Processing: 100%|████████████████████████████████████████████| 20000/20000 [01:06<00:00, 302.49it/s]


In [47]:
len(rx)

20480

In [48]:
print ("Now placing Nb")
for count in tqdm(np.arange(natoms_summed[0],natoms_summed[1]), desc="Processing", ncols=100):
    x,y,z = box*ran.random(),box*ran.random(),box*ran.random()
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off+0.5 for value in dist):
            checker = "Passed"
        else:
            x,y,z = box*ran.random(),box*ran.random(),box*ran.random()
    rx.append(x)
    ry.append(y)
    rz.append(z)

Now placing Nb


Processing: 100%|█████████████████████████████████████████████| 20000/20000 [07:05<00:00, 47.05it/s]


In [49]:
print ("Now placing O")
for count in tqdm(np.arange(natoms_summed[1],natoms_summed[2]), desc="Processing", ncols=100):
    x,y,z = box*ran.random(),box*ran.random(),box*ran.random()
    checker = "Failed"
    while checker != "Passed":
        dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
        if all(value > cut_off+0.3 for value in dist):
            checker = "Passed"
        else:
            x,y,z = box*ran.random(),box*ran.random(),box*ran.random()
    rx.append(x)
    ry.append(y)
    rz.append(z)

Now placing O


Processing: 100%|███████████████████████████████████████████| 60000/60000 [6:45:49<00:00,  2.46it/s]


In [53]:
out1 = open("RT_seed.POSCAR", 'w')
print ("comment line",file=out1)
print ("%16.10f"%(1.0),file=out1)
print ("%23.16f"%(box),"%22.16f"%(0.0),"%22.16f"%(0.0),file=out1)
print ("%23.16f"%(0.0),"%22.16f"%(box),"%22.16f"%(0.0),file=out1)
print ("%23.16f"%(0.0),"%22.16f"%(0.0),"%22.16f"%(box),file=out1)
print ("   Li","   Nb","    O",file=out1)

n_Li_seed = int(len(seed)*0.2)
n_Nb_seed = n_Li_seed
n_O_seed = int(3*n_Li_seed)

print ("   ",n_Li_seed,"   ",n_Nb_seed,"    ",n_O_seed,file=out1)
print ("Cartesian",file=out1)

    
for ct in range(len(seed)):
    print ("%20.16f"%(rx[ct]),"%20.16f"%(ry[ct]),"%20.16f"%(rz[ct]),file=out1)
    
out1.close()

In [54]:
out2 = open("LiNbO3_random_100k.POSCAR", 'w')
print ("comment line",file=out2)
print ("%16.10f"%(1.0),file=out2)
print ("%23.16f"%(box),"%22.16f"%(0.0),"%22.16f"%(0.0),file=out2)
print ("%23.16f"%(0.0),"%22.16f"%(box),"%22.16f"%(0.0),file=out2)
print ("%23.16f"%(0.0),"%22.16f"%(0.0),"%22.16f"%(box),file=out2)
print ("   Li","   Nb","    O",file=out2)
print ("   ",n_Li,"   ",n_Nb,"    ",n_O,file=out2)
print ("Cartesian",file=out2)

    
for ct in range(len(seed),len(rx)):
    print ("%20.16f"%(rx[ct]),"%20.16f"%(ry[ct]),"%20.16f"%(rz[ct]),file=out2)
    
out2.close()